In [319]:
import numpy as np
import pandas as pd
from datetime import datetime as dt, timedelta as tmd

from warnings import filterwarnings
filterwarnings('ignore')

from importlib import reload
import xAPIConnector.xAPIConnector
reload(xAPIConnector.xAPIConnector)
from xAPIConnector.xAPIConnector import *


import xAPIConnector.DataLoader
reload(xAPIConnector.DataLoader)
from xAPIConnector.DataLoader import *

import xAPIConnector.config
reload(xAPIConnector.config)
from xAPIConnector.config import user_id, pwd

In [323]:
symbols = ['BITCOIN', 'ETHEREUM']
start, interval = '2024-12-01 00:00:00', '5min'

dl = DataLoader(user_id, pwd)
data = dl.getData(symbols=symbols, start_date=start, interval=interval)

[2025-01-13 14:25:07.698361] Loguję do API...
	Wysyłam zapytanie do API...
	Wysyłam zapytanie do API...
[2025-01-13 14:25:14.685136] Wylogowuję z API...


Klasyfikatory można budować na mnóstwo różnych sposobów. Ogólnie trzeba się zdecydować, na ilu obserwacjach wstecz ma się opierać klasyfikacja. Liczbę tych obserwacji nazwiemy 'window' i przyjmiemy jako parametr.

Na razie ustawiamy klasyfikację na podstawie poprzednich 4 godzin.

Pierwszy pomysł to klasyfikacja na podstawie stóp zwrotu, klasy również będziemy budować na podstawie stóp zwrotu. Metoda budowania klas zostanie omówiona dalej.

In [324]:
window = int(4*60/int(interval[:-3]))
returnsBTC = data.loc[:, 'BITCOIN'].pct_change().dropna()

In [325]:
def generate_X_y(data: pd.DataFrame | pd.Series, window: int, skip: int) -> tuple:   
    # można ustawić okna zachodzące (0 < skip < window)
    # można ustawić niezachodzące (skip >= window)
    assert skip > 0, "Pętla w kodzie nigdy się nie zakończy..."
    
    # Generujemy 'okna'
    X = pd.DataFrame(columns=range(window-1))
    y = pd.Series()
    
    i = len(data)
    count = 0
    while i >= window:
        temp_y = data.iloc[i-window:i]
        
        X.loc[count, :] = temp_y.iloc[:-1].values
        y.loc[count] = temp_y.iloc[-1]
        
        i = i - skip    
        count += 1
        
    return np.array(X), np.array(y)

### Regresja logistyczna

In [326]:
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import LeaveOneOut

Pierwszy pomysł na zdefiniowanie klas to przyjęcie '1' jeśli stopa zwrotu jest $>threshold$ oraz '0' w p.p.

Próg odcięcia można podać jako parametr i już po pierwszych testach widać, że dla prostej klasyfikacji typu "czy zwrot > 0" skuteczność jest około $50\%$. Im wyższy próg, tym lepsza skuteczność.

In [327]:
skip = window
X, y = generate_X_y(data=returnsBTC, window=window, skip=skip)

threshold = 0.001
y = (np.array(y) > threshold)*1.0

clf = LogisticRegression().fit(X, y)
print(f"Skuteczność treningowa modelu: {clf.score(X, y):.4%}")

Skuteczność treningowa modelu: 79.8969%


Teraz podzielimy dane na treningowe i testowe, po czym sprawdzimy jakość predykcji

In [328]:
train_test_ratio = 0.7
len_train = int(train_test_ratio*X.shape[0])

Xtrain = X[:len_train, :]
ytrain = y[:len_train]

Xtest = X[len_train:, :]
ytest = y[len_train:]

print(f"[INFO] W zbiorze treningowym znaduje się {Xtrain.shape[0]} obserwacji.")
print(f"[INFO] W zbiorze testowym znajduje się {Xtest.shape[0]} obserwacji.")

[INFO] W zbiorze treningowym znaduje się 135 obserwacji.
[INFO] W zbiorze testowym znajduje się 59 obserwacji.


In [329]:
clf = LogisticRegression().fit(Xtrain, ytrain)
print(f"Skuteczność treningowa modelu: {clf.score(Xtrain, ytrain):.4%}")
print(f"Skuteczność testowa modelu: {clf.score(Xtest, ytest):.4%}")

Skuteczność treningowa modelu: 80.0000%
Skuteczność testowa modelu: 79.6610%
